# 📊 Heisenberg Chain: Convergence Study

Full DMRG analysis on the spin-1/2 Heisenberg XXX chain with scaling in system size and bond dimension.

In [ ]:
!rm -rf PyTenNet && git clone https://github.com/tigantic/PyTenNet.git
import sys; sys.path.insert(0, 'PyTenNet')

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tensornet import MPS, heisenberg_mpo, dmrg

torch.manual_seed(42)
print(f'PyTorch {torch.__version__}')

## 1. Energy vs Bond Dimension (χ scaling)

In [ ]:
L = 20
chi_values = [8, 16, 32, 64, 128]
energies = []

H = heisenberg_mpo(L=L, J=1.0)

for chi in chi_values:
    psi = MPS.random(L=L, d=2, chi=chi)
    psi, E, _ = dmrg(psi, H, num_sweeps=20, chi_max=chi)
    energies.append(E)
    print(f'χ={chi:3d}: E = {E:.10f}, E/L = {E/L:.10f}')

# Extrapolate to χ→∞
E_exact_approx = energies[-1]  # Best estimate
print(f'
Best energy (χ=128): {E_exact_approx:.10f}')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Energy vs chi
ax1.plot(chi_values, energies, 'o-', linewidth=2, markersize=8, color='navy')
ax1.set_xlabel('Bond dimension χ', fontsize=12)
ax1.set_ylabel('Ground state energy', fontsize=12)
ax1.set_title(f'Heisenberg L={L}: Energy Convergence', fontsize=14)
ax1.grid(True, alpha=0.3)

# Error vs 1/chi (extrapolation)
E_ref = energies[-1]
errors = [abs(E - E_ref) for E in energies[:-1]]
inv_chi = [1/c for c in chi_values[:-1]]
ax2.loglog(chi_values[:-1], errors, 's-', linewidth=2, markersize=8, color='crimson')
ax2.set_xlabel('Bond dimension χ', fontsize=12)
ax2.set_ylabel('|E - E_best|', fontsize=12)
ax2.set_title('Truncation Error Scaling', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Finite-Size Scaling

In [ ]:
L_values = [10, 16, 20, 30, 40, 50]
E_per_site = []

for L in L_values:
    H = heisenberg_mpo(L=L, J=1.0)
    psi = MPS.random(L=L, d=2, chi=64)
    psi, E, _ = dmrg(psi, H, num_sweeps=15, chi_max=64)
    E_per_site.append(E/L)
    print(f'L={L:2d}: E/L = {E/L:.8f}')

# Bethe ansatz: E/L → 1/4 - ln(2) ≈ -0.4431 as L→∞
E_bethe = 0.25 - np.log(2)
print(f'
Bethe ansatz (L→∞): E/L = {E_bethe:.8f}')

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(L_values, E_per_site, 'o-', linewidth=2, markersize=8, label='DMRG')
plt.axhline(y=E_bethe, color='red', linestyle='--', linewidth=2, label=f'Bethe ansatz = {E_bethe:.4f}')
plt.xlabel('System size L', fontsize=12)
plt.ylabel('Energy per site E/L', fontsize=12)
plt.title('Finite-Size Scaling: Heisenberg Chain', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Entanglement Entropy Profile

In [ ]:
L = 40
H = heisenberg_mpo(L=L, J=1.0)
psi = MPS.random(L=L, d=2, chi=128)
psi, E, _ = dmrg(psi, H, num_sweeps=20, chi_max=128)

entropies = [psi.entropy(bond).item() for bond in range(L-1)]

# CFT prediction: S(x) = (c/6) * ln((L/π) * sin(πx/L)) + const
# c = 1 for Heisenberg
c = 1.0
x = np.arange(1, L)
S_cft = (c/6) * np.log((L/np.pi) * np.sin(np.pi * x / L))
S_cft = S_cft - S_cft.min() + min(entropies)  # Shift to match

plt.figure(figsize=(10, 5))
plt.plot(range(1, L), entropies, 'o', markersize=6, label='DMRG', alpha=0.8)
plt.plot(x, S_cft, '-', linewidth=2, color='red', label='CFT (c=1)')
plt.xlabel('Bond position', fontsize=12)
plt.ylabel('Entanglement entropy S', fontsize=12)
plt.title(f'Entanglement Profile (L={L}, χ=128)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'Max entropy (center): {max(entropies):.4f}')

## 4. Convergence Benchmark

In [ ]:
import time

L = 30
chi = 64
H = heisenberg_mpo(L=L, J=1.0)
psi = MPS.random(L=L, d=2, chi=chi)

energies = []
times = []

start = time.time()
for sweep in range(1, 26):
    psi, E, _ = dmrg(psi, H, num_sweeps=1, chi_max=chi)
    elapsed = time.time() - start
    energies.append(E)
    times.append(elapsed)
    if sweep <= 5 or sweep % 5 == 0:
        print(f'Sweep {sweep:2d}: E = {E:.10f} ({elapsed:.2f}s)')

print(f'
Total time: {times[-1]:.2f}s')
print(f'Time per sweep: {times[-1]/25:.3f}s')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Energy convergence
axes[0].plot(range(1, 26), energies, 'o-', color='darkgreen')
axes[0].set_xlabel('Sweep')
axes[0].set_ylabel('Energy')
axes[0].set_title('Energy Convergence')
axes[0].grid(True, alpha=0.3)

# Energy change
dE = [abs(energies[i] - energies[i-1]) for i in range(1, len(energies))]
axes[1].semilogy(range(2, 26), dE, 's-', color='crimson')
axes[1].set_xlabel('Sweep')
axes[1].set_ylabel('|ΔE|')
axes[1].set_title('Convergence Rate')
axes[1].grid(True, alpha=0.3)

# Time scaling
axes[2].plot(range(1, 26), times, 'D-', color='purple')
axes[2].set_xlabel('Sweep')
axes[2].set_ylabel('Cumulative time (s)')
axes[2].set_title('Runtime')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()